# Modelo predictivo

### Importar paquetes

In [70]:
import pandas as pd
import numpy as np
import os
import re

### Limpieza de los datos

Empezaremos por todas las transformaciones necesarias tras el análisis previo. El objetivo es conseguir un conjunto de entrenamiento y test para nuestros modelos.

A pesar de que el reto específica que se nos proporcionarían diferentes conjuntos de test, tan solo contamos con uno. Eso nos hizo sospechar, tras un examen manual, nos dimos cuenta que los lotes de test están mezclados con el resto. Así que lo primero será separar dichos lotes y a partir de ahi construir la tabla con los campos relevantes. 

Este proceso será iterativo, por lo que complica la entrega, ya que solo podemos entregar un archivo. Por tanto se irá documentando, las diferentes transformaciones que se harán en distintos puntos del proceso hasta conseguir un modelo con buen rendimiento.

Como primer acercamiento será obtener los valores agregados de las distintas fases para poder predecir la variable objetivo.
Para ello necesitaremos crear un `pipeline` que garantize el orden de los pasos, dado que la entrada que se nos proporciona para el test es insuficiente para realizar una predicción

In [2]:
df_cultivo_test = pd.read_excel('raw_data/Fases producción_test v02.xlsx', sheet_name='Cultivo final', engine='openpyxl')
df_cultivo_test.head(5)

,LOTE,Orden en el encadenado,LOTE parental,ID Bioreactor,Fecha/hora inicio,Fecha/hora fin,Volumen de inóculo utilizado,Turbidez inicio cultivo,Turbidez fin cultivo,Viabilidad final cultivo,ID Centrífuga,Centrifugación 1 turbidez,Centrifugación 2 turbidez,Producto 1,Producto 2
0,24054,1,NaN,14616,2024-04-16 08:12:00,2024-04-18 07:28:00,81.6,15.44,85.60,184800000,14246.0,27.84,23.96,NaN,NaN
1,24055,1,NaN,14614,2024-04-13 08:18:00,2024-04-15 08:30:00,NaN,14.32,73.68,175200000,12912.0,30.96,23.16,NaN,NaN
2,24056,1,NaN,14615,2024-04-13 08:18:00,2024-04-15 08:15:00,NaN,14.56,82.40,168000000,14246.0,29.52,28.88,NaN,NaN
3,24057,1,NaN,13170,2024-04-16 08:12:00,2024-04-18 07:41:00,82.4,17.76,78.96,180800000,12912.0,31.04,25.32,NaN,NaN
4,24058,2,24055.0,14614,2024-04-15 12:28:00,2024-04-17 08:14:00,87.2,18.00,82.40,144800000,12912.0,26.08,20.36,NaN,NaN


In [3]:
lotes_test = df_cultivo_test['LOTE'].unique()
len(lotes_test)

56

In [4]:
df_cultivo = pd.read_excel('raw_data/Fases producción v02.xlsx', sheet_name='Cultivo final', engine='openpyxl')
df_cultivo.head(5)

,LOTE,Orden en el encadenado,LOTE parental,ID Bioreactor,Fecha/hora inicio,Fecha/hora fin,Volumen de inóculo utilizado,Turbidez inicio cultivo,Turbidez fin cultivo,Viabilidad final cultivo,ID Centrífuga,Centrifugación 1 turbidez,Centrifugación 2 turbidez,Producto 1,Producto 2
0,23019,1,NaN,14615,2023-03-21 07:30:00,2023-03-23 06:30:00,82.4,17.28,91.2,184000000,17825,NaN,NaN,1747.920,6.00
1,23020,1,NaN,14616,2023-03-21 07:30:00,2023-03-23 06:30:00,80.4,18.80,91.2,181600000,14246,NaN,NaN,1676.160,6.56
2,23021,1,NaN,13170,2023-03-22 07:30:00,2023-03-24 06:30:00,66.4,16.16,86.4,248000000,17825,NaN,NaN,1928.496,8.08
3,23022,1,NaN,14614,2023-03-22 07:30:00,2023-03-24 06:30:00,85.6,18.48,83.2,229600000,12912,NaN,NaN,1782.800,5.92
4,23023,1,NaN,14615,2023-03-28 07:27:00,2023-03-30 10:00:00,77.6,17.12,74.4,132800000,17825,26.56,20.88,1861.840,2.96


___

In [80]:
def get_orden_data():
    # Cargamos datos
    df = pd.read_excel('raw_data/OF 123456 v02.xlsx', engine='openpyxl')

    # Corregimos caracteres especiales en Lote 
    df['LOTE'] = df['Lote'].apply(lambda x: int(x.replace('/', '').replace('P', '')))

    # Renombrado de columnas
    df = df.rename(columns={'Orden': 'orden', 'Cantidad entregada': 'cantidad'})
    
    # LOTE como indice y variables escogidas
    df = df.set_index('LOTE')
    final_cols = ['orden', 'cantidad']

    return df[final_cols]

In [81]:
def get_preinoculo_data(lotes):
    # Carga de datos
    df = pd.read_excel('raw_data/Fases producción v02.xlsx', sheet_name='Preinóculo', header=[0, 1], na_values=['NA', 'N.A'], engine='openpyxl')
    # Corregir cabeceras
    new_cols = [l2 if 'Unnamed' in l1 else f"{l1}-{l2}" for l1, l2 in df.columns.to_list()]
    df.columns = new_cols

    # Filtro de lotes
    df = df[df['LOTE'].isin(lotes)]

    # Imputación valores perdidos por la media
    for c in df.select_dtypes(include='float').columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')
        df[c] = df[c].fillna(df[c].mean())
        
    # Valores de pH y turbidez seleccionados para siguiente fase
    ph_cols = [col for col in df.columns if col.startswith('pH')]
    turbidez_cols = [col for col in df.columns if col.startswith('Turbidez')]
    lineas_cols = [col for col in df.columns if col.startswith('Línea')]

    # el ph y turbidez finales las consideramos como la mezcla de las dos líneas escogidas
    df['ph'] = (
        (
            df[ph_cols].values *
            df[lineas_cols].values
        ).sum(axis=1) /
        df[lineas_cols].sum(axis=1)
    ).round(3)
    
    df['turbidez'] = (
        (
            df[turbidez_cols].values *
            df[lineas_cols].values
        ).sum(axis=1) /
        df[lineas_cols].sum(axis=1)
    ).round(2)

    # Calculo duración
    # 1. obtenemos las horas
    # 2. corregimos las fechas "al reves"
    # 3. corregimos las horas negativas
    ini_col = 'Fecha/hora inicio'
    fin_col = 'Fecha/hora fin'
    df['duracion_horas'] = (df[fin_col] - df[ini_col]).dt.total_seconds() / 3600
    df.loc[df['duracion_horas'] < 0, [ini_col, fin_col]] = df.loc[df['duracion_horas'] < 0, [fin_col, ini_col]].values
    df['duracion_horas'] = df['duracion_horas'].abs().round(2)

    # LOTE como indice y variables escogidas
    df = df.set_index('LOTE')
    final_cols = ['ph','turbidez','duracion_horas']

    return df[final_cols]


In [82]:
def get_inoculo_data(lotes, df_preinoculo):
    # Carga de datos
    df = pd.read_excel('raw_data/Fases producción v02.xlsx', sheet_name='Inóculo', engine='openpyxl')
    df['Viabilidad final cultivo'] = pd.to_numeric(df['Viabilidad final cultivo'], errors='coerce')

    # Filtrado de lotes 
    df = df[df['LOTE'].isin(lotes)]
    df = df.dropna(subset=['Fecha/hora inicio','Fecha/hora fin'])

    # Renombrado de columnas
    df = df.rename(columns={'Fecha/hora inicio': 'ts_inicio', 
                            'Fecha/hora fin': 'ts_fin', 
                            'ID bioreactor': 'id_bioreactor',
                            'Turbidez inicio cultivo': 'turbidez_ini',
                            'Turbidez final culttivo': 'turbidez_fin',
                            'Viabilidad final cultivo': 'viabilidad'}
    )

    # Imputación de valores perdidos
    df.loc[df['turbidez_ini'].isna(), ['turbidez_ini']] = (
        df[df['turbidez_ini'].isna()]
        .join(df_preinoculo, 
              on='LOTE', how='left', rsuffix='preinoculo')
        ['turbidez']
    )
    df['turbidez_dif'] = df['turbidez_fin'] - df['turbidez_ini']

    # > Si con la anterior imputación volvemos a tener un NaN en la resta, imputamos por la media
    df['turbidez_dif'] = df['turbidez_dif'].fillna(df['turbidez_dif'].mean())
    
    # Calculo duracion de la fase en horas
    ini_col = 'ts_inicio'
    fin_col = 'ts_fin'
    df['duracion_horas'] = (df[fin_col] - df[ini_col]).dt.total_seconds() / 3600
    df.loc[df['duracion_horas'] < 0, [ini_col, fin_col]] = df.loc[df['duracion_horas'] < 0, [fin_col, ini_col]].values
    df['duracion_horas'] = df['duracion_horas'].abs().round(2)

    # LOTE como indice y variables escogidas
    df = df.set_index('LOTE')
    final_cols = ['id_bioreactor', 'ts_inicio', 'ts_fin', 'turbidez_dif', 'viabilidad', 'duracion_horas']

    return df[final_cols]

In [83]:
def get_cultivo_data(lotes):

    # Carga de datos
    df = pd.read_excel('raw_data/Fases producción v02.xlsx', sheet_name='Cultivo final', engine='openpyxl')
    df['Viabilidad final cultivo'] = pd.to_numeric(df['Viabilidad final cultivo'], errors='coerce')

    # Filtrado de lotes 
    df = df[df['LOTE'].isin(lotes)]

    # Renombrado de columnas
    df = df.rename(columns={'Fecha/hora inicio': 'ts_inicio', 
                            'Fecha/hora fin': 'ts_fin', 
                            'ID Bioreactor': 'id_bioreactor',
                            'ID Centrífuga': 'id_centrifugadora',
                            'LOTE parental': 'lote_padre',
                            'Volumen de inóculo utilizado': 'volumen_ini',
                            'Turbidez inicio cultivo': 'turbidez_ini',
                            'Turbidez fin cultivo': 'turbidez_fin',
                            'Centrifugación 1 turbidez':'turbidez_cfg1',
                            'Centrifugación 2 turbidez':'turbidez_cfg2',
                            'Viabilidad final cultivo': 'viabilidad',
                            'Producto 1': 'producto'}
    )

    # Imputar valores perdidos
    df['lote_padre'] = df['lote_padre'].fillna(0)
    df['lote_padre'] = df['lote_padre'].astype(int)
    # La variable volumen del dataset de inoculo no tiene misma semantica, se imputa por la media (no hay demasiada desviacion)
    df['volumen_ini'] = df['volumen_ini'].fillna(df['volumen_ini'].mean())

    # NO ME GUSTA ESTA IDEA
    df['turbidez_cfg1'] = df['turbidez_cfg1'].fillna(df['turbidez_cfg1'].mean())
    df['turbidez_cfg2'] = df['turbidez_cfg2'].fillna(df['turbidez_cfg2'].mean())

    # Calculos de diferencias
    df['turbidez_dif'] = (df['turbidez_fin'] - df['turbidez_ini']).round(2) # la turbidez aumenta en los bioreactores
    df['turbidez_cfg_dif'] = (df['turbidez_cfg1'] - df['turbidez_cfg2']).round(2)  # la turbidez disminuye tras cada centrifugado


    # Calculo duracion de la fase en horas
    ini_col = 'ts_inicio'
    fin_col = 'ts_fin'
    df['duracion_horas'] = (df[fin_col] - df[ini_col]).dt.total_seconds() / 3600
    df.loc[df['duracion_horas'] < 0, [ini_col, fin_col]] = df.loc[df['duracion_horas'] < 0, [fin_col, ini_col]].values
    df['duracion_horas'] = df['duracion_horas'].abs().round(2)

    # LOTE como indice y variables escogidas
    df = df.set_index('LOTE')
    final_cols = ['id_bioreactor', 'id_centrifugadora', 'lote_padre', 'ts_inicio', 'ts_fin', 
                  'volumen_ini', 'turbidez_dif', 'turbidez_cfg_dif', 'duracion_horas', 'producto']

    return df[final_cols]
    

In [84]:
def concatenate_bioreactor_data():
    bioreactores_files = [f for f in os.listdir('./raw_data') if not os.path.isdir(f) and f.startswith('Biorreactor')]
    bioreactores_dfs = []
    for bio_fname in bioreactores_files:
        
        numbers = re.findall(r'\d+', bio_fname)
        id_bioreactor = int(numbers[0])

        df = pd.read_excel(f'./raw_data/{bio_fname}', sheet_name='Datos', index_col='DateTime', parse_dates=True, engine='openpyxl')
        
        clean_cols = [col.split('.')[1].lower() for col in df.columns]
        renamed_cols = {old: new for old, new in zip(df.columns, clean_cols)}
        
        df = df.rename(columns=renamed_cols)
        df['id_bioreactor'] = id_bioreactor
        bioreactores_dfs.append(df)

    df = pd.concat(bioreactores_dfs, axis=0).sort_index().set_index('id_bioreactor', append=True)

    # SAVE en lugar de cargarlo siempre al vuelo ?!
    return df

In [110]:
concatenate_bioreactor_data() # revisar los indices !!

agitation_pv  air_sparge_pv  \
DateTime   id_bioreactor                                
2023-03-15 14618                  72.0       0.000000   
           14614                  80.0       0.000000   
           14617                   0.0       0.000000   
           14615                  80.0       0.000000   
           14616                  80.0       0.000000   
...                                ...            ...   
2024-09-11 13172                   0.0       0.000000   
           14614                   0.0       0.000000   
           13171                  20.0       0.000000   
           13169                  80.0       0.000000   
           13170                  80.0      64.313885   

                          biocontainer_pressure_pv    do_1_pv    do_2_pv  \
DateTime   id_bioreactor                                                   
2023-03-15 14618                          1.869612  15.953993        NaN   
           14614                          0.572660   0.000000  -0.005530   
           14617                        480.000000   0.000000        NaN   
           14615                          0.268969  18.732030   0.000000   
           14616                          0.715311  16.557993   0.000000   
...                                            ...        ...        ...   
2024-09-11 13172                         -0.579450   0.000000   0.000000   
           14614                        480.000000   0.000000   0.000000   
           13171                         -2.052007   0.000000   0.000000   
           13169                         -0.399304   0.000000  16.229234   
           13170                          2.397993  14.085248   0.000000   

                          gas_overlay_pv  load_cell_net_pv     ph_1_pv  \
DateTime   id_bioreactor                                                 
2023-03-15 14618                4.000057            169.60 -527.387158   
           14614                4.000087           1576.80   -0.156925   
           14617                0.000000              0.00    1.707277   
           14615                3.999874           1636.80    5.914182   
           14616                3.999639           1652.80    5.929625   
...                                  ...               ...         ...   
2024-09-11 13172                4.000006             -1.44    1.444136   
           14614                0.000000            -17.60    1.415245   
           13171                3.999893            161.60    5.072766   
           13169                3.999885           1660.00    5.874518   
           13170                4.003011           1664.00    5.878986   

                             ph_2_pv  pump_1_pv  pump_1_total  pump_2_pv  \
DateTime   id_bioreactor                                                   
2023-03-15 14618            5.998478        0.0    104.159973   0.000000   
           14614            5.888288        0.0     14.880000   0.000000   
           14617           -1.969051        0.0     49.599991   0.000000   
           14615           -0.234763        0.0     22.439996   0.000000   
           14616         -389.260962        0.0     39.679996   0.000000   
...                              ...        ...           ...        ...   
2024-09-11 13172           -0.613304        0.0      0.000000   0.000000   
           14614           -0.172975        0.0     59.519989   0.000000   
           13171           -0.016351        0.0      7.439999   0.000000   
           13169           -0.347235        0.0     29.760001   7.182158   
           13170           -0.164859        0.0     32.400000   0.971887   

                          pump_2_total  single_use_do_pv  single_use_ph_pv  \
DateTime   id_bioreactor                                                     
2023-03-15 14618              0.000000         16.713527          6.016000   
           14614            191.200293        799.991992        799.967969   
           14617           2014

In [85]:
def concatentate_cfg_data():
    centrifugadora_files = [f for f in os.listdir('./raw_data') if not os.path.isdir(f) and f.startswith('Centrífuga')]
    centrifugadora_dfs = []
    cols_name_map = {
        'CTF0101.EN_Parcial' : 'n_descargas_parciales',
        'CTF0101.EN_Total': 'n_descargas_totales',
        'D01780551.PV': 'valvula_apertura',
        'D01906041.PV': 'caudal',
        'D01916047.PV': 'contrapresion',
        'D01916503.PV': 'presion_agua',
        'D01919022.PV': 'velocidad_separacion'
    }
    for cfg_fname in centrifugadora_files:
        
        numbers = re.findall(r'\d+', cfg_fname)
        id_centrifugadora = int(numbers[0])

        df = pd.read_excel(f'./raw_data/{cfg_fname}', sheet_name='Datos', index_col='DateTime', parse_dates=True, engine='openpyxl')

        new_cols = df.columns.map(lambda x: x.replace(f'{id_centrifugadora}_',''))
        df = df.rename(columns={old_col: new_col for old_col, new_col in zip(df.columns, new_cols)})
        df = df.rename(columns=cols_name_map)
        df['id_centrifugadora'] = id_centrifugadora

        centrifugadora_dfs.append(df)

    df = pd.concat(centrifugadora_dfs, axis=0).sort_index().set_index('id_centrifugadora', append=True)

    # SAVE en lugar de cargarlo siempre al vuelo ?! 
    return df

In [130]:
concatentate_cfg_data()

velocidad_separacion  presion_agua  \
DateTime            id_centrifugadora                                       
2023-03-15 00:00:00 17825                               0.0           NaN   
                    12912                               0.0           NaN   
                    14246                               0.0           NaN   
2023-03-15 00:15:00 17825                               0.0           NaN   
                    12912                               0.0           NaN   
...                                                     ...           ...   
2024-09-10 23:45:00 17825                               0.0      0.471475   
                    12912                               0.0      0.410748   
2024-09-11 00:00:00 12912                               0.0      0.411111   
                    17825                               0.0      0.471162   
                    14246                               0.0      0.136927   

                                       contrapresion  caudal  \
DateTime            id_centrifugadora                          
2023-03-15 00:00:00 17825                   0.411225     0.0   
                    12912                   0.427143     0.0   
                    14246                   0.426557     0.0   
2023-03-15 00:15:00 17825                   0.411225     0.0   
                    12912                   0.424826     0.0   
...                                              ...     ...   
2024-09-10 23:45:00 17825                   0.467852     0.0   
                    12912                   0.420372     0.0   
2024-09-11 00:00:00 12912                   0.420197     0.0   
                    17825                   0.467651     0.0   
                    14246                   0.008479     0.0   

                                       valvula_apertura  n_descargas_totales  \
DateTime            id_centrifugadora                                          
2023-03-15 00:00:00 17825                         100.0                  0.0   
                    12912                         100.0                  0.0   
                    14246                         100.0                  0.0   
2023-03-15 00:15:00 17825                         100.0                  0.0   
                    12912                         100.0                  0.0   
...                                                 ...                  ...   
2024-09-10 23:45:00 17825                         100.0                  0.0   
                    12912                         100.0                  0.0   
2024-09-11 00:00:00 12912                         100.0                  0.0   
                    17825                         100.0                  0.0   
                    14246                         100.0                  0.0   

                                       n_descargas_parciales  
DateTime            id_centrifugadora                         
2023-03-15 00:00:00 17825                                0.0  
                    12912                                0.0  
                    14246                                0.0  
2023-03-15 00:15:00 17825                                0.0  
                    12912                                0.0  
...                                                      ...  
2024-09-10 23:45:00 17825                                0.0  
                    12912                                0.0  
2024-09-11 00:00:00 12912                                0.0  
                    17825                                0.0  
                    14246                                0.0  

[157239 rows x 7 columns]

In [86]:
def get_cfg_time_data():

    # Carga de datos
    df = pd.read_excel('raw_data/Horas inicio fin centrífugas.xlsx', parse_dates=["DATEVALUE"] , engine='openpyxl')

    df = df.rename(columns={
        'EQUIPO': 'id_centrifugadora',
        'Orden': 'orden',
        'Operación': 'operacion',
        'DATEVALUE': 'ts'
    })
    df['operacion'] = df['operacion'].map({'Centrifugació 1 ini': 'cfg_1_ini', 
                                           'Centrifugació 1 fi':'cfg_1_fin', 
                                           'Centrifugació 2 ini': 'cfg_2_ini', 
                                           'Centrifugació 2 fi': 'cfg_2_fin'
                                           }
    )

    df_pivot = df.pivot(columns='operacion', values=['ts'])
    df_pivot.columns = list(map("_".join, df_pivot.columns))

    index_cols = ['id_centrifugadora', 'orden']
    df = df[index_cols]
    df = pd.concat([df, df_pivot], axis=1)
    df = df.groupby(by=index_cols).first()

    df['duracion_cfg_1'] = ((df['ts_cfg_1_fin'] - df['ts_cfg_1_ini']).dt.total_seconds() / 3600).round(2)
    df['duracion_cfg_2'] = ((df['ts_cfg_2_fin'] - df['ts_cfg_2_ini']).dt.total_seconds() / 3600).round(2)

    return df


In [113]:
df = get_cfg_time_data()
df.loc[(df.index.get_level_values('orden') == 200179217) ]

,,ts_cfg_1_fin,ts_cfg_1_ini,ts_cfg_2_fin,ts_cfg_2_ini,duracion_cfg_1,duracion_cfg_2
id_centrifugadora,orden,,,,,,
14246,200179217,2023-03-23 10:26:25,2023-03-23 08:00:49,2023-03-23 13:34:10,2023-03-23 12:11:04,2.43,1.38


In [178]:
def get_ipc_data(lotes):

    # Cargar datos 
    df_cultivo = pd.read_excel('raw_data/Cinéticos IPC.xlsx', sheet_name='Cultivos finales', engine='openpyxl')
    df_cfg = pd.read_excel('raw_data/Cinéticos IPC.xlsx', sheet_name='Centrifugación', engine='openpyxl')

    df_cultivo['Turbidez'] = pd.to_numeric(df_cultivo['Turbidez'], errors='coerce')
    df_cultivo['Glucosa g/L'] = pd.to_numeric(df_cultivo['Glucosa g/L'], errors='coerce')

    df_cfg['Turbidez'] = pd.to_numeric(df_cfg['Turbidez'], errors='coerce')

    # Filtrar lotes
    df_cultivo['LOTE'] = df_cultivo['Lote'].apply(lambda l: int(l.replace('P','')) if type(l) != int else l)
    df_cfg['LOTE'] = df_cfg['Lote'].apply(lambda l: int(l.replace('P','')) if type(l) != int else l)

    df_cultivo = df_cultivo[df_cultivo['LOTE'].isin(lotes)]
    df_cfg = df_cfg[df_cfg['LOTE'].isin(lotes)]

    # Agrupación de datos
    df_cultivo = (
        df_cultivo
        .groupby(by='LOTE', as_index=False)
        .agg({'Fecha': ['count'], 'Turbidez': ['mean', np.ptp], 'Viabilidad': ['mean', np.ptp], 'Glucosa g/L': ['mean', np.ptp]})
        .set_index('LOTE')
    )
    df_cultivo.columns = [ '_'.join(str(s).lower() for s in col) for col in df_cultivo.columns]

    df_cfg = (
        df_cfg
        .groupby(by=['LOTE', 'Centrífuga', 'Centrifugada (1 o 2)'])
        .agg(turbidez_cfg_last=('Turbidez', 'last')) # mean # max_vol=('Volumen centrifugado (L)', 'max'), 
        .reset_index()
        .rename(columns={'Centrífuga': 'id_centrifugadora'})
    )

    df_pivot = df_cfg.pivot(columns='Centrifugada (1 o 2)', values=['turbidez_cfg_last']) # 'max_vol', 
    df_pivot.columns = [ '_'.join(str(s) for s in col) for col in df_pivot.columns]

    df_cfg = pd.concat([df_cfg, df_pivot], axis=1).drop(['Centrifugada (1 o 2)', 'turbidez_cfg_last'], axis=1) # 'max_vol',

    df_cfg = df_cfg.groupby(by=['LOTE', 'id_centrifugadora'], as_index=False).first().set_index('LOTE')
    
    df = df_cultivo.merge(df_cfg, on='LOTE')
    return df


In [179]:
get_ipc_data(lotes)

,fecha_count,turbidez_mean,turbidez_ptp,viabilidad_mean,viabilidad_ptp,glucosa g/l_mean,glucosa g/l_ptp,id_centrifugadora,turbidez_cfg_last_1,turbidez_cfg_last_2
LOTE,,,,,,,,,,
23019,10,46.352000,73.92,9.498000e+07,1.770000e+08,1.439760,1.7624,17825,28.16,21.52
23020,10,46.992000,72.40,4.833216e+08,4.028811e+09,1.471280,1.6312,14246,26.88,12.88
23021,11,40.378182,70.24,9.388606e+07,2.361600e+08,1.556582,1.7576,17825,28.96,20.24
23022,11,40.538182,64.72,1.020812e+08,2.209467e+08,1.604218,1.6816,12912,26.56,9.44
23023,12,45.846667,64.48,1.057858e+08,2.061973e+08,1.416933,1.7328,17825,26.88,20.64
...,...,...,...,...,...,...,...,...,...,...
24105,3,62.900000,66.32,1.098873e+08,1.700507e+08,1.228000,0.9840,17825,31.44,18.72
24106,4,64.960000,60.40,1.010267e+08,1.534400e+08,1.156000,1.3840,12912,32.64,26.00
24107,4,64.780000,66.24,1.023267e+08,1.727733e+08,1.070000,1.2320,14246,28.32,23.84


In [75]:
df_inoculo = pd.read_excel('raw_data/Cinéticos IPC.xlsx', sheet_name='Inóculos', engine='openpyxl')
df = df_inoculo.groupby(by='Lote').agg({'Turbidez': ['mean', np.ptp], 'Viabilidad': ['mean', np.ptp]})
df.corr()

Turbidez           Viabilidad          
                     mean       ptp       mean       ptp
Turbidez   mean  1.000000  0.672656   0.566922  0.259499
           ptp   0.672656  1.000000   0.107991  0.585533
Viabilidad mean  0.566922  0.107991   1.000000  0.333276
           ptp   0.259499  0.585533   0.333276  1.000000

In [110]:
df_info_general = get_orden_data()
lotes = df_info_general.index.unique()

df_preinoculo = get_preinoculo_data(lotes)
df_inoculo = get_inoculo_data(lotes, df_preinoculo)

df_cultivo = get_cultivo_data(lotes)

In [180]:
df_cultivo

,id_bioreactor,id_centrifugadora,lote_padre,ts_inicio,ts_fin,volumen_ini,turbidez_dif,turbidez_cfg_dif,duracion_horas,producto
LOTE,,,,,,,,,,
23019,14615,17825,0,2023-03-21 07:30:00,2023-03-23 06:30:00,82.40,73.92,6.44,47.00,1747.920
23020,14616,14246,0,2023-03-21 07:30:00,2023-03-23 06:30:00,80.40,72.40,6.44,47.00,1676.160
23021,13170,17825,0,2023-03-22 07:30:00,2023-03-24 06:30:00,66.40,70.24,6.44,47.00,1928.496
23022,14614,12912,0,2023-03-22 07:30:00,2023-03-24 06:30:00,85.60,64.72,6.44,47.00,1782.800
23023,14615,17825,0,2023-03-28 07:27:00,2023-03-30 10:00:00,77.60,57.28,5.68,50.55,1861.840
...,...,...,...,...,...,...,...,...,...,...
24049,14617,12912,0,2024-03-16 09:22:00,2024-03-18 08:23:00,83.60,53.76,13.56,47.02,1342.800
24050,14614,6379,0,2024-03-23 08:57:00,2024-03-25 08:28:00,84.16,49.84,2.80,47.52,1422.800
24051,13169,12912,0,2024-03-23 08:57:00,2024-03-25 08:33:00,84.16,63.04,14.12,47.60,1486.560


In [111]:
df_info_general

,orden,cantidad
LOTE,,
23019,200178572,13.80
23020,200179217,13.60
23021,200181620,13.50
23022,200182428,13.80
23023,200182429,13.70
...,...,...
24108,200205686,13.63
24106,200205695,13.69
24107,200205696,13.45


In [72]:
df_cultivo

,id_bioreactor,id_centrifugadora,lote_padre,ts_inicio,ts_fin,volumen_ini,turbidez_dif,turbidez_cfg_dif,duracion_horas,producto
LOTE,,,,,,,,,,
23019,14615,17825,0,2023-03-21 07:30:00,2023-03-23 06:30:00,82.40,73.92,6.44,47.00,1747.920
23020,14616,14246,0,2023-03-21 07:30:00,2023-03-23 06:30:00,80.40,72.40,6.44,47.00,1676.160
23021,13170,17825,0,2023-03-22 07:30:00,2023-03-24 06:30:00,66.40,70.24,6.44,47.00,1928.496
23022,14614,12912,0,2023-03-22 07:30:00,2023-03-24 06:30:00,85.60,64.72,6.44,47.00,1782.800
23023,14615,17825,0,2023-03-28 07:27:00,2023-03-30 10:00:00,77.60,57.28,5.68,50.55,1861.840
...,...,...,...,...,...,...,...,...,...,...
24049,14617,12912,0,2024-03-16 09:22:00,2024-03-18 08:23:00,83.60,53.76,13.56,47.02,1342.800
24050,14614,6379,0,2024-03-23 08:57:00,2024-03-25 08:28:00,84.16,49.84,2.80,47.52,1422.800
24051,13169,12912,0,2024-03-23 08:57:00,2024-03-25 08:33:00,84.16,63.04,14.12,47.60,1486.560
